# Projeto 1 - Informed and adversarial search strategies
## Inteligência Artificial
### Grupo: 2
### Turma: PL5

- Diogo Filipe Gonçalves Cruz (202005947)
- Michel Castro Costa e Silva (202200437)
- Rodrigo Luz de Carvalho (20200184)

# Problema - Connect 4

**Connect Four** é um jogo de estratégia para dois jogadores. É jogado com 42 peças (21 de uma cor e 21 de outra) numa grade vertical com 7 colunas. Cada coluna pode conter no máximo 6 peças. Os dois jogadores jogam alternadamente colocando uma ficha numa das colunas. Quando uma ficha é colocada numa coluna, ela cai até atingir a parte inferior ou a ultima ficha nessa coluna. Um jogador vence quando pelo menos quatro de suas fichas estão alinhadas numa linha, coluna ou diagonal.

## Objetivo
O objetivo deste trabalho é implementar um algoritmo **A*** e **Monte Carlo Tree Search** (MCTS) para jogar este jogo. 

In [ ]:
#
%pip install numpy

import numpy as np
import os #poder usar função clear
from sys import platform # para poder determinar qual o sistema operacional.
import copy

if platform == "win32": # Se for windows
    clear = lambda: os.system('cls') 
else: # Caso seja unix based
    clear = lambda: os.system('clear')


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy in c:\users\michel\scoop\apps\python\current\lib\site-packages (1.26.2)




[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


A celúla acima instala e importa a biblioteca numpy, necessaria pois usamos arrays do numpy para representar a board

In [ ]:
# Definição de algumas constantes usadas durante o código
NUM_ROW = 6
NUM_COL = 7
EMPTY = "-"
PLAYER_PIECE = "X"
AI_PIECE = "O"

In [ ]:
def get_segments(board):
        segments = []
        # Verifica linhas e colunas
        for i in range(NUM_ROW):
            line = board[i]
            for j in range(4):
                segments.append(line[j : j + 4])
            col = board[:, i]
            for j in range(3):
                segments.append(col[j : j + 4])

        # Verifica a ultima coluna
        col = board[:, NUM_COL - 1]
        for j in range(3):
            segments.append(col[j : j + 4])

        # Verifica as diagonais principais
        for i in range(-2, 4):
            dia = np.diag(board, i)
            for j in range(len(dia) - 3):
                segments.append(dia[j : j + 4])

        # Dá flip no array e verifica as diagonais principais do array flipado
        # (equivalentes às diagonais perpendiculares às principais do array original)
        state_tr = np.fliplr(board)
        for i in range(-2, 4):
            dia = np.diag(state_tr, i)
            for j in range(len(dia) - 3):
                segments.append(dia[j : j + 4])
        return segments

def gen_dict():
    board = [[f"{i},{j}" for j in range(7)]for i in range(6)]
    board = np.array(board)
    segments = get_segments(board)
    index_dict = {}
    position_dict = {}
    for j in range(7):
        for i in range(6):
            temp = []
            for z,segment in enumerate(segments):
                if f"{i},{j}" in segment:
                    temp.append(z)
                index_dict[(i,j)] = temp
                position_dict[(i,j)] = [[tuple(map(int, pos.split(","))) for pos in segments[index]] for index in temp]
    return index_dict,position_dict
index_dict, pos_dict = gen_dict()

{(0, 0): [0, 4, 48], (1, 0): [4, 5, 7, 46], (2, 0): [4, 5, 6, 14, 45], (3, 0): [4, 5, 6, 21, 68], (4, 0): [5, 6, 28, 67], (5, 0): [6, 35, 65], (0, 1): [0, 1, 11, 51], (1, 1): [7, 8, 11, 12, 48, 49], (2, 1): [11, 12, 13, 14, 15, 46, 47, 68], (3, 1): [11, 12, 13, 21, 22, 45, 66, 67], (4, 1): [12, 13, 28, 29, 64, 65], (5, 1): [13, 35, 36, 62], (0, 2): [0, 1, 2, 18, 54], (1, 2): [7, 8, 9, 18, 19, 51, 52, 68], (2, 2): [14, 15, 16, 18, 19, 20, 48, 49, 50, 66, 67], (3, 2): [18, 19, 20, 21, 22, 23, 46, 47, 63, 64, 65], (4, 2): [19, 20, 28, 29, 30, 45, 61, 62], (5, 2): [20, 35, 36, 37, 59], (0, 3): [0, 1, 2, 3, 25, 56, 68], (1, 3): [7, 8, 9, 10, 25, 26, 54, 55, 66, 67], (2, 3): [14, 15, 16, 17, 25, 26, 27, 51, 52, 53, 63, 64, 65], (3, 3): [21, 22, 23, 24, 25, 26, 27, 48, 49, 50, 60, 61, 62], (4, 3): [26, 27, 28, 29, 30, 31, 46, 47, 58, 59], (5, 3): [27, 35, 36, 37, 38, 45, 57], (0, 4): [1, 2, 3, 32, 66], (1, 4): [8, 9, 10, 32, 33, 56, 63, 64], (2, 4): [15, 16, 17, 32, 33, 34, 54, 55, 60, 61, 62

O código acima gera dois dicionarios usados no calcúlo das heuristicas de segmentos da board, ambos tem como chave uma posição da board, já os valores são:
- index_dict: Os indices dos segmentos na lista de heuristica, que terão suas heuristicas alteradas ao colocar uma peça nessa posição.
- pos_dict: Uma lista das listas de posições contidas no segmentos que contem a posição chave
abaixo tem um exemplo de ambos dicionarios

In [13]:
print(index_dict[(1,2)])
print(pos_dict[(1,2)])

[7, 8, 9, 18, 19, 51, 52, 68]
[[(1, 0), (1, 1), (1, 2), (1, 3)], [(1, 1), (1, 2), (1, 3), (1, 4)], [(1, 2), (1, 3), (1, 4), (1, 5)], [(0, 2), (1, 2), (2, 2), (3, 2)], [(1, 2), (2, 2), (3, 2), (4, 2)], [(0, 1), (1, 2), (2, 3), (3, 4)], [(1, 2), (2, 3), (3, 4), (4, 5)], [(0, 3), (1, 2), (2, 1), (3, 0)]]


# A star

A heurística utilizada no A* foi a heurística fornecida no protocolo do trabalho. 
Se houver vencedor num estado o valor da heurística é:
- +525 se o vencedor for "X"
- -525 se o vencedor for "O"

Se não houver vencedor, o valor da heurística é o somatório do valor de todas as sequências de 4 peças, atribuindo-se os seguintes valores:
- 3 "X" -> +100
- 2 "X" -> +10
- 1 "X" -> +1
- Sem peças ou com peças dos dois jogadores -> 0
- 1 "O" -> -1
- 2 "O" -> -10
- 3 "O" -> -100

Somando-se um bonús:

- +15 se for a vez de "X" jogar
- -15 se for a vez de "O" jogar

A seguir temos a definição da classe game, que representa um jogo, nela se tem os atributos:
- game_winner: armazena quem ganhou o jogo ou a peça vazia, caso ninguem tenha ganhado
- board_is_full: boolean que fica verdadeiro quando todas as colunas estão cheias
- segment_heuristics: lista da heuristica de cada segmento
- self.board: array que representa o tabuleiro
- self.last_move: guarda quem jogou por ultimo


In [ ]:
#
import numpy as np
from dictgeneration import gen_dict
NUM_ROW = 6
NUM_COL = 7
EMPTY = "-"
PLAYER_PIECE = "X"
AI_PIECE = "O"
index_dict,pos_dict = gen_dict()


class game:
    def __init__(self, calculate_heuristics=True):
        self.game_winner = EMPTY  # variáveis que controlam o fim do jogo
        self.board_is_full = False
        self.segment_heuristics = [0 for _ in range (69)]
        self.board = np.full([NUM_ROW, NUM_COL], EMPTY)
        self.last_move = None
        self.calculate_heuristics = calculate_heuristics
        self.turn = 0

    def drawBoard(self):
        for i in range(7): print(i, end=" ") #imprime os numeros das colunas
        print() # coloca newline
        for line in np.flip(self.board, 0):
            for piece in line:
                print(piece, end=" ")
            print()# coloca newline
        return

    """Dado um input, verifica a função availableCollumns para saber se é válido.
    Se não for pede novamente o input, do contrário usa a função putGamePiece para alterar a board."""

    def playOneTurn(self):
        available = self.availableCollumns()
        try:  # error handling
            collumn = int(
                input("Choose in which collumn do you wanna play or press 9 to quit: ")
            )
        except:
            collumn = -1

        while collumn not in available:
            if collumn == 9:
                quit()  # opção para quitar
            try:  # more error handlings
                collumn = int(
                    input(
                        "The collumn you selected is either full or invalid, choose another one or press 9 to quit: "
                    )
                )
            except:
                collumn = -1
        self.putGamePiece(collumn, PLAYER_PIECE)
        return

    """Verifica qual a próxima row vazia e alterar a põe a peça nesta row. 
    Checa após cada movimento a função check_win_after_move para saber se houve ganhador."""

    def putGamePiece(self, collumn, piece):
        available = self.availableCollumns()
        if collumn not in available:
            print("The last move was invalid, below is the last state of the board and the move made, the game will quit")
            self.drawBoard()
            print("last move = {collumn}")
            quit()

        piece_placement = self.nextEmptyRowinCollumn(collumn)
        self.board[piece_placement][collumn] = piece
        if self.check_win_after_move(piece_placement, collumn, piece):
            self.game_winner = piece  # checar se houve ganhador
        elif not self.availableCollumns():
            self.board_is_full = True  # se não há colunas vazias, a board está cheia
        # not list aparentemente é um dos jeitos mais eficientes de checar se uma lista está vazia, python é estranho - M
        if self.calculate_heuristics: self.update_heuristics(piece_placement, collumn) # atualiza as heuristicas se o bool for True
        self.last_move = collumn
        self.turn += 1
        return

    """Checa a última row para saber quais colunas não estão cheias. Retorna a lista de colunas."""

    def availableCollumns(self):
        available = []
        for i, value in enumerate(self.board[NUM_ROW - 1]):
            if value == EMPTY:
                available.append(i)
        return available

    """Dado uma coluna, verifica qual a próxima row vazia. Retorna o número da row."""

    def nextEmptyRowinCollumn(self, collumn):
        for i, value in enumerate(self.board[:, collumn]):
            if value == EMPTY:
                return i
        return

    """Função que checa se houve vitória após cada movimento. 
    Verifica somente os arrays que contém a peça em [move_row, move_col]."""
    def check_win_after_move(self, move_row, move_col, piece):
        win_segment = [piece, piece, piece, piece]
        for i in self.segments_that_intersect(move_row, move_col): # verifica se houve vitoria em todos os segmentos que são modificados pelo ultimo movimento
            if i == win_segment: return True
        return False

    # retorna lista com todos os segmentos de 4 em todas as direções
                
    def get_segments(self):
        segments = []
        # Verifica linhas e colunas
        for i in range(NUM_ROW):
            line = self.board[i]
            for j in range(4):
                segments.append(line[j : j + 4])
            col = self.board[:, i]
            for j in range(3):
                segments.append(col[j : j + 4])

        # Verifica a ultima coluna
        col = self.board[:, NUM_COL - 1]
        for j in range(3):
            segments.append(col[j : j + 4])

        # Verifica as diagonais principais
        for i in range(-2, 4):
            dia = np.diag(self.board, i)
            for j in range(len(dia) - 3):
                segments.append(dia[j : j + 4])

        # Dá flip no array e verifica as diagonais principais do array flipado
        # (equivalentes às diagonais perpendiculares às principais do array original)
        state_tr = np.fliplr(self.board)
        for i in range(-2, 4):
            dia = np.diag(state_tr, i)
            for j in range(len(dia) - 3):
                segments.append(dia[j : j + 4])
        return segments
    def segments_that_intersect(self, move_row, move_col):
        segments = [[self.board[pos[0],pos[1]] for pos in list] for list in pos_dict[(move_row,move_col)]]
        return segments

    # avalia um segmento e retorna a sua pontuação
    def update_heuristics(self,move_row,move_col):
        indexes = index_dict[(move_row,move_col)]
        segments = self.segments_that_intersect(move_row, move_col)             
        for i in range(len(indexes)):
            self.segment_heuristics[indexes[i]] = self.evaluate(segments[i])

    def evaluate(self, segment):
        count_x = 0
        count_o = 0

        for i in segment:
            if i == PLAYER_PIECE:
                count_x += 1
            elif i == AI_PIECE:
                count_o += 1

        if (count_x == 0 and count_o == 0) or (count_x > 0 and count_o > 0):
            return 0

        match count_x:
            case 1:
                return 1
            case 2:
                return 10
            case 3:
                return 50
            case 4:
                return 512

        match count_o:
            case 1:
                return -1
            case 2:
                return -10
            case 3:
                return -50
            case 4:
                return -512

    # avalia todos as posições para descobrir se há vencedor
    def evaluate_all(self):
        if self.game_winner == PLAYER_PIECE:
            return 512
        elif self.game_winner == AI_PIECE:
            return -512
        elif self.board_is_full:
            return 0
        else:
            sum = 0
            if self.player() == PLAYER_PIECE:
                sum = sum + 16
            elif self.player() == AI_PIECE:
                sum = sum - 16
            for segment in self.get_segments():
                sum = sum + self.evaluate(segment)
            return sum

    """Verifica se um determinado estado é um estado terminal/final 
    (estado em que um dos jogadores ganhou ou em que não há ações possíveis)"""

    def terminal(self):
        if self.game_winner == PLAYER_PIECE or self.game_winner == AI_PIECE:
            return True
        else:
            return self.board_is_full
        """
        elif len(self.availableCollumns())==0:
            return True
        else:
            return False
        """

    """Recebe um estado e retorna o vencedor (no caso deste existir)"""

    def checkwin_wholeboard(self):
        for (
            segment
        ) in self.get_segments():  # Itera sobre todos os segmentos de tamanho 4
            if np.array_equal(segment, ["X", "X", "X", "X"]):
                return "X"
            elif np.array_equal(segment, ["O", "O", "O", "O"]):
                return "O"
        return None

    """Otimização da função player. Utiliza self.turn para avaliar qual o jogador atual.
    Não está em uso em outras classes."""

    def player(self):
        if self.board_is_full:
            return None
        elif self.turn % 2 == 0:
            return PLAYER_PIECE
        else:
            return AI_PIECE

    """Recebe um estado e retorna o jogador nesse turno."""

    # def player(self):
    #     cx = 0  # contador de X
    #     co = 0  # contador de O
    #     for i in range(6):
    #         a = self.board[i]
    #         for j in range(7):
    #             b = a[j]
    #             if b == PLAYER_PIECE:
    #                 cx += 1
    #             elif b == AI_PIECE:
    #                 co += 1
    #     # caso do tabuleiro estar completamente ocupado
    #     if cx + co == NUM_COL * NUM_ROW:
    #         return None

        # Se o número de X's for menor ou igual ao numéro de O's, então é a vez de X jogar.
        # if cx <= co:
        #     return PLAYER_PIECE
        # else:
        #     return AI_PIECE

    def utility(self):
        if self.game_winner == PLAYER_PIECE:
            return 512
        elif self.game_winner == AI_PIECE:
            return -512
        else:
            return 0


# A star

A heurística utilizada no A* foi a heurística fornecida no protocolo do trabalho. 
Se houver vencedor num estado o valor da heurística é:
- +525 se o vencedor for "X"
- -525 se o vencedor for "O"

Se não houver vencedor, o valor da heurística é o somatório do valor de todas as sequências de 4 peças, atribuindo-se os seguintes valores:
- 3 "X" -> +100
- 2 "X" -> +10
- 1 "X" -> +1
- Sem peças ou com peças dos dois jogadores -> 0
- 1 "O" -> -1
- 2 "O" -> -10
- 3 "O" -> -100

Somando-se um bonús:

- +15 se for a vez de "X" jogar
- -15 se for a vez de "O" jogar

# MCTS

Na implementação do MCTS foi utilizado o algoritmo UCT (Upper Confidence Bound for Trees) para a seleção do nó a ser expandido. O valor de C foi ajustado manualmente de forma a otimizar os resultados.

# Extras - MiniMax e Alpha-Beta Pruning

Foi também implementado o algoritmo de MiniMax e Alpha-Beta Pruning para melhorar comparação de resultados. Neste algoritmo usamos a mesma heurística do A*.
